**Objetivo:** Liste `employees.name`, `departments.name` (como `department`) para cada funcionário.
- Join: employees.departmentId = departments.id

In [1]:
import pandas as pd
import duckdb as db

departments = pd.read_csv('../Dataset/Exercicios/departments.csv')

departments.head()

,id,name
0,1,Engineering
1,2,HR
2,3,Sales
3,4,Finance
4,5,Support


In [2]:
employees = pd.read_csv('../Dataset/Exercicios/employees.csv')

employees.head()

,id,name,salary,managerId,departmentId,email
0,1,Ana,9000,NaN,1,ana@corp.com
1,2,Bruno,8500,1.0,1,bruno@corp.com
2,3,Carla,8500,1.0,1,carla@corp.com
3,4,Diego,7800,2.0,1,diego@corp.com
4,5,Elisa,7200,2.0,1,elisa@corp.com


In [3]:
df = db.query(
    """
    select 
        t1.name as employee,
        t2.name as department

    from read_csv_auto('../Dataset/Exercicios/employees.csv') as t1
    left join read_csv_auto('../Dataset/Exercicios/departments.csv') as t2 on t2.id = t1.departmentId
              
    """).to_df()

df

,employee,department
0,Ana,Engineering
1,Bruno,Engineering
2,Carla,Engineering
3,Diego,Engineering
4,Elisa,Engineering
5,Fabio,HR
6,Gabi,HR
7,Henrique,HR
8,Iara,Sales
9,Joao,Sales


Second Highest Salary (Medium)
**Objetivo:** Retorne o 2º maior salário distinto da tabela `employees` (coluna `salary`).
- Se não existir, retorne NULL.

In [4]:
df = db.query(
    """
    with employees as (
    select 
        name,
        dense_rank() over (order by salary desc) as rn,
        salary
    from read_csv_auto('../Dataset/Exercicios/employees.csv') as t1
    )
    
    select 
        *
    from employees
    where rn = 2
             
    """).to_df()

df

,name,rn,salary
0,Nina,2,9500


 Nth Highest Salary (Medium)
**Objetivo:** Crie uma query (ou função, se seu SGBD permitir) que retorne o N-ésimo maior salário distinto de `employees`.
- Teste com N=1,2,3,4.

In [5]:
df = db.query(
    """
    with employees as (
        select distinct
            salary, 
            dense_rank() over(order by salary desc) as rn
        from read_csv_auto('../Dataset/Exercicios/employees.csv') as t1
    )
    select * from employees order by rn
    
             
    """).to_df()

df

,salary,rn
0,11000,1
1,9500,2
2,9300,3
3,9000,4
4,8500,5
5,7800,6
6,7200,7
7,6500,8
8,6200,9
9,5900,10


Rank Scores (Medium)
**Tabela:** `scores(id, score)`
**Objetivo:** Retorne `score` e o `rank` denso (dense rank) em ordem decrescente.
- Mesma nota -> mesmo rank; não “pula” números.

In [10]:
df = db.query(
    """
    with scores as (
        select 
            score,
            dense_rank() over(order by score desc) as rn

        from read_csv_auto('../Dataset/Exercicios/scores.csv') as t1
    )
    select * from scores 
    
             
    """).to_df()

df

,score,rn
0,100,1
1,98,2
2,98,2
3,90,3
4,88,4
5,88,4
6,70,5
7,60,6


 Consecutive Numbers (Medium)
**Tabela:** `logs(id, num)`
**Objetivo:** Encontre números que aparecem **pelo menos 3 vezes consecutivas** (por id crescente).
- Retorne apenas os números.

In [12]:
df = db.query(
    """
    with scores as (
        select 
      *
        from read_csv_auto('../Dataset/Exercicios/logs.csv') as t1
    )
    select * from scores 
    
             
    """).to_df()

df

,id,num
0,1,1
1,2,1
2,3,1
3,4,2
4,5,1
5,6,2
6,7,2
7,8,2
8,9,3
9,10,3
